In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# set the max columns to none
pd.set_option('display.max_columns', None)

import tensorflow as tf
from tensorflow import keras

## Data Hotel Goibibo 

In [25]:
df_goibibo = pd.read_csv('goibibo_com-travel_sample.csv')
# select potential parameter only
df_goibibo.drop(columns=['additional_info','province','area','locality','crawl_date','hotel_brand','hotel_category','pageurl','qts','query_time_stamp','room_area','similar_hotel','sitename','uniq_id','image_count'], inplace=True)
# drop data review
df_goibibo.drop(columns=['review_count_by_category','site_review_count','site_stay_review_rating','guest_recommendation'], inplace=True)

df_goibibo.head(3)

,address,city,country,hotel_description,hotel_facilities,hotel_star_rating,latitude,longitude,point_of_interest,property_id,property_name,property_type,room_count,room_facilities,room_type,site_review_rating,state
0,"15th Mile, N.H.21,Manali, District Kullu,Himac...",Manali,India,The standard check-in time is 12:00 PM and the...,Doctor on Call|Dry Cleaning|Laundry Service Av...,2,32.139387,77.154660,Hadimba Temple|Naggar Village|Himalayan Nyingm...,HTLGBO1000016238,Baragarh Regency,Resort,17,Room Service |Basic Bathroom Amenities|Cable /...,Deluxe Room,4.0,Himachal Pradesh
1,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...",Gurgaon,India,The standard check-in time is 12:00 PM and the...,Airport Transfer Available / Surcharge|Banquet...,0,28.472097,77.072546,Sushant Lok|Sahara Mall|Amity International Sc...,HTLGBO1000015822,Asian Suites A- 585,Guest House,18,Room Service |Air Conditioning |Basic Bathroom...,Deluxe Room With Free WIFI,4.5,Haryana
2,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...",Goa,India,The standard check-in time is 12:00 PM and the...,Swimming Pool|Bar / Lounge |Laundry Service Av...,0,15.548398,73.757634,"Anjuna Beach|Calangute Beach|Titos lane, baga|...",HTLGBO1000004551,Bevvan Resort,Resort,15,Room Service |Air Conditioning |Cable / Satell...,Standard Room,2.5,Goa


In [26]:
# eksplorasi dataset
print('jumlah property id: ',len(df_goibibo['property_id'].unique()))
print('jumlah property name: ',len(df_goibibo['property_name'].unique()))
print('jumlah property type: ',len(df_goibibo['property_type'].unique()))
print('jumlah data jika drop null: ',len(df_goibibo.dropna()))
df_goibibo.dropna(inplace=True)

jumlah property id:  4000
jumlah property name:  3805
jumlah property type:  18
jumlah data jika drop null:  2030


In [28]:
# df_goibibo.drop_duplicates(subset=['property_name'], inplace=True)

print('cek jumlah null dari tiap parameter: \n')
print('property_name: ', df_goibibo['property_name'].isna().sum())
print('property_type: ', df_goibibo['property_type'].isna().sum())
print('property_id: ', df_goibibo['property_id'].isna().sum())
print('hotel_description: ', df_goibibo['hotel_description'].isna().sum())
print('address: ', df_goibibo['address'].isna().sum())
print('city: ', df_goibibo['city'].isna().sum())
print('state: ', df_goibibo['state'].isna().sum())
print('country: ', df_goibibo['country'].isna().sum())
print('latitude: ', df_goibibo['latitude'].isna().sum())
print('longitude: ', df_goibibo['longitude'].isna().sum())
print('point_of_interest: ', df_goibibo['point_of_interest'].isna().sum())
print('hotel_facilities: ', df_goibibo['hotel_facilities'].isna().sum())
print('hotel_star_rating: ', df_goibibo['hotel_star_rating'].isna().sum())
print('room_count: ', df_goibibo['room_count'].isna().sum())
print('room_facilities: ', df_goibibo['room_facilities'].isna().sum())
print('room_type: ', df_goibibo['room_type'].isna().sum())

print('site_review_rating: ', df_goibibo['site_review_rating'].isna().sum())

cek jumlah null dari tiap parameter: 

property_name:  0
property_type:  0
property_id:  0
hotel_description:  0
address:  0
city:  0
state:  0
country:  0
latitude:  0
longitude:  0
point_of_interest:  0
hotel_facilities:  0
hotel_star_rating:  0
room_count:  0
room_facilities:  0
room_type:  0
site_review_rating:  0


In [29]:
print('Daftar tipe properti: \n',df_goibibo['property_type'].unique())
print('Total kota: ',len(df_goibibo['city'].unique()))
print('Total provinsi: ',len(df_goibibo['state'].unique()))
print('Daftar hotel rating: ',len(df_goibibo['hotel_star_rating'].unique()), df_goibibo['hotel_star_rating'].unique())


Daftar tipe properti: 
 ['Resort' 'Guest House' 'Cottage' 'Hotel' 'Homestay' 'Palace'
 'Service Apartment' 'Hostel' 'Lodge' 'Tent' 'Villa' 'Houseboat'
 'Bungalow' 'BnB' 'Beach Hut' 'Motel' 'Farm Stay']
Total kota:  343
Total provinsi:  27
Daftar hotel rating:  6 [2 0 3 1 4 5]


In [30]:
df_goibibo

,address,city,country,hotel_description,hotel_facilities,hotel_star_rating,latitude,longitude,point_of_interest,property_id,property_name,property_type,room_count,room_facilities,room_type,site_review_rating,state
0,"15th Mile, N.H.21,Manali, District Kullu,Himac...",Manali,India,The standard check-in time is 12:00 PM and the...,Doctor on Call|Dry Cleaning|Laundry Service Av...,2,32.139387,77.154660,Hadimba Temple|Naggar Village|Himalayan Nyingm...,HTLGBO1000016238,Baragarh Regency,Resort,17,Room Service |Basic Bathroom Amenities|Cable /...,Deluxe Room,4.0,Himachal Pradesh
1,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...",Gurgaon,India,The standard check-in time is 12:00 PM and the...,Airport Transfer Available / Surcharge|Banquet...,0,28.472097,77.072546,Sushant Lok|Sahara Mall|Amity International Sc...,HTLGBO1000015822,Asian Suites A- 585,Guest House,18,Room Service |Air Conditioning |Basic Bathroom...,Deluxe Room With Free WIFI,4.5,Haryana
2,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...",Goa,India,The standard check-in time is 12:00 PM and the...,Swimming Pool|Bar / Lounge |Laundry Service Av...,0,15.548398,73.757634,"Anjuna Beach|Calangute Beach|Titos lane, baga|...",HTLGBO1000004551,Bevvan Resort,Resort,15,Room Service |Air Conditioning |Cable / Satell...,Standard Room,2.5,Goa
3,Simsa,Manali,India,The standard check-in time is 12:00 PM and the...,Doctor on Call|Dry Cleaning|Laundry Service Av...,2,32.223603,77.185900,The Mall|Tibetian Monastery|Vashisht Hot Baths...,HTLGBO1000007993,Apple Inn Cottage,Cottage,24,Basic Bathroom Amenities|Cable / Satellite / P...,Deluxe Room,5.0,Himachal Pradesh
4,"8180 Street No.-6,Arakashan Road,Paharganj",Delhi,India,The standard check-in time is 12:00 PM and the...,Internet Access - Surcharge|Laundry Service Av...,2,28.646777,77.212735,Gaffar Market|YMCA Institute|Agrasen Boali|Cha...,HTLGBO1000008281,Anmol Hotel Pvt.Ltd,Hotel,20,Basic Bathroom Amenities|Cable / Satellite / P...,Standard Room Non AC,2.8,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3992,"sarvaritu vilas , inside udaipole , opp sagasj...",Udaipur,India,The standard check-in time is 12:00 PM and the...,Airport Transfer Available / Surcharge|Doctor ...,2,24.575598,73.696959,Jagdish Temple|City Palace|Udaipur Bus Stand|U...,HTLGBO1000012898,Hotel Shubh Laxmi,Hotel,44,Room Service |Basic Bathroom Amenities|Cable /...,Deluxe Non A/c Room,4.2,Rajasthan
3993,"Near Yasholaxmi Nagar, Kolhapur Rd,",Ichalkaranji,India,The standard check-in time is 10:00 AM and the...,Bar / Lounge |Doctor on Call|Dry Cleaning|Inte...,2,16.690042,74.456394,Yasholaxmi Nagar,HTLGBO1000083318,Hotel Signature,Hotel,8,Room Service |Air Conditioning |Alarm Clock|Ca...,Suite Room,5.0,Maharashtra
3994,"Near Railway Station , Kota Jn",Kota,India,The standard check-in time is 12:00 PM and the...,Bar / Lounge |Doctor on Call|Laundry Service A...,2,25.222694,75.878170,Kota Airport|Station Road Area,HTLGBO1000009343,Hotel Shree Anand Restaurant &amp; Bar,Hotel,16,Room Service |Air Conditioning |Basic Bathroom...,AC ROOM,3.4,Rajasthan
3997,"Marol Maroshi Road, Near To Marol naka FireBri...",Mumbai,India,The standard check-in time is 12:00 PM and the...,Banquet Facilities|Bar / Lounge |Currency Exch...,3,19.110926,72.879038,Mount Mary Church|Juhu Beach|Domestic Airport|...,HTLGBO1000001584,Hotel Silver Inn,Hotel,41,Air Conditioning |Cable / Satellite / Pay TV a...,Gold Room,4.0,Maharashtra
